# Segmenting and Clustering Neighborhoods in Toronto - PART 1

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: / 

In [ ]:
# Import Data from Wikipedia
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wikiRequest = requests.get(URL).text
soup = BeautifulSoup(wikiRequest, 'html.parser')

PostalCodesTable = soup.find('table', class_='wikitable')

In [ ]:
# Parse the data into the Dataframe
postalCodesdf = pd.DataFrame()

for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['td'])
    try:
        postalCode = data[0].text
        borough = data[1].text
        neighborhood = data[2].text
        postalCode= postalCode.replace('\n','')
        borough=borough.replace('\n', '')
        neighborhood=neighborhood.replace('\n', '')
        neighborhood=neighborhood.replace(' / ', ', ')
        if(neighborhood=='Not assigned'):
            neighborhood=borough
    except IndexError:pass
    
    if(borough!='Not assigned'):
        postalCodesRow = pd.Series([postalCode,borough,neighborhood])
        postalCodes = pd.DataFrame([postalCodesRow])
        postalCodesdf = pd.concat([postalCodesdf, postalCodes], ignore_index=True)
        
postalCodesdf = postalCodesdf.rename(columns={0:'Postal Code',1:'Borough',2:'Neighborhood'})
postalCodesdf.head()

# Segmenting and Clustering Neighborhoods in Toronto - PART 2

In [ ]:
# Add location Coordinates of all the Postal Codes
geodf = pd.read_csv('http://cocl.us/Geospatial_data', names=['Postal Code','Latitude','Longittude'],skiprows=1)
geodf.head()
postalcCodesdf=pd.merge(postalCodesdf, geodf, on='Postal Code')
postalcCodesdf.head()

# Segmenting and Clustering Neighborhoods in Toronto - PART 3

In [ ]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#Working with only Downtown Toronto
torontodf = postalcCodesdf[postalcCodesdf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
torontodf.head()

In [ ]:
address = 'Downtown Toronto,Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# Define Foursquare Credentials and Version
LIMIT =100
radius = 500
CLIENT_ID = '0IQXIUQBL5CK51XCMZUB0QLIDVRAWBUD4YTIJGFWZOGGO1FL' # your Foursquare ID
CLIENT_SECRET = '2JPLPR2TXS5ILNTAULTR3L5SDMHDHCQZWEJOP3EJ1CTQUMYT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# type your answer here

downtownVenues = getNearbyVenues(names=torontodf['Neighborhood'],
                                   latitudes=torontodf['Latitude'],
                                   longitudes=torontodf['Longittude']
                                  )


In [ ]:
downtownVenues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(downtownVenues['Venue Category'].unique())))

In [ ]:
# one hot encoding
downtownOnehot = pd.get_dummies(downtownVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtownOnehot['Neighborhood'] = downtownVenues['Neighborhood'] 

downtownOnehot.set_index('Neighborhood', inplace=True)
# # move neighborhood column to the first column
# fixed_columns = [downtownOnehot.columns[-1]] + list(downtownOnehot.columns[:-1])
# downtownOnehot = downtownOnehot[fixed_columns]

downtownOnehot.head()

In [ ]:
downtownOnehot.shape
downtownGrouped = downtownOnehot.groupby('Neighborhood').mean().reset_index()
downtownGrouped
downtownGrouped.shape

In [ ]:
num_top_venues = 5

for hood in downtownGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtownGrouped[downtownGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtownGrouped['Neighborhood']

for ind in np.arange(downtownGrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtownGrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

downtownGroupedClustering = downtownGrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtownGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtownMerged = torontodf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtownMerged = downtownMerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtownMerged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtownMerged['Latitude'], downtownMerged['Longittude'], downtownMerged['Neighborhood'], downtownMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters